In [5]:
from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w", encoding="utf-8") as f:
        f.write(persona["persona"])


In [6]:
from llama_index.core import SimpleDirectoryReader
reader=SimpleDirectoryReader(input_dir='data')
documents = reader.load_data()
len(documents)

5000

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
pipeline=IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5')]
)
nodes = await pipeline.arun(documents=documents[:10])
nodes

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\ParsRayaneh\anaconda3\envs\LLM\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ParsRayaneh\AppData\Local\llama_index\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[TextNode(id_='81846331-167c-4f73-ad58-eb2766572812', embedding=[-0.004853234626352787, -0.0013557710917666554, 0.029231207445263863, 0.002511759288609028, 0.006844346411526203, -0.04035452380776405, -0.01567406766116619, 0.0008557089604437351, -0.06685124337673187, -0.057179953902959824, 0.0023792560677975416, -0.002592533128336072, -0.019007418304681778, 0.02505875565111637, -0.012189099565148354, 0.008104939013719559, 0.01530382689088583, 0.0902668684720993, -0.005911828950047493, 0.007389557082206011, -0.006306863855570555, -0.052323486655950546, 0.06532450020313263, -0.020926987752318382, -0.03870146721601486, 0.012293187901377678, 0.012286021374166012, -0.01158346701413393, -0.05233351141214371, -0.13911202549934387, -0.04689991474151611, -0.0012807019520550966, 0.0016607559518888593, 0.004190206527709961, 0.0552629679441452, -0.00017188434139825404, 0.005021648481488228, 0.07657596468925476, 0.004718500189483166, 0.018224557861685753, 0.027601055800914764, 0.0025517207104712725,

In [9]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

10

In [11]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model)

In [18]:
from llama_index.llms.ollama import Ollama 

In [25]:
llm=Ollama(model='qwen3:8b',request_timeout=650.0)

In [28]:
import nest_asyncio

nest_asyncio.apply()  # This is needed to run the query engine

query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
#response = query_engine.query(
#    "Respond using a persona that describes author and travel experiences?"
#)
#response

In [29]:
from llama_index.core.evaluation import FaithfulnessEvaluator


# query index
evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query(
    "What battles took place in New York City in the American Revolution?"
)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

True